In [34]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, LSTM, GRU, Dense, RepeatVector, TimeDistributed, Dropout
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [35]:
def generate_data(n_samples, sequence_length, vocab_size):
    X = np.random.randint(1, vocab_size, size=(n_samples, sequence_length))
    Y = np.flip(X, axis=1)
    return X, Y

n_samples = 10000
sequence_length = 10
vocab_size = 20

X, Y = generate_data(n_samples, sequence_length, vocab_size)
X_one_hot = tf.keras.utils.to_categorical(X, num_classes=vocab_size)
Y_one_hot = tf.keras.utils.to_categorical(Y, num_classes=vocab_size)

X_train, X_val, y_train, y_val = train_test_split(X_one_hot, Y_one_hot, test_size=0.2)

In [36]:
def build_seq2seq(cell_type='RNN'):
    model = Sequential()
    if cell_type == 'RNN':
        model.add(SimpleRNN(256, input_shape=(sequence_length, vocab_size)))
        model.add(RepeatVector(sequence_length))
        model.add(SimpleRNN(256, return_sequences=True))
    elif cell_type == 'LSTM':
        model.add(LSTM(256, input_shape=(sequence_length, vocab_size)))
        model.add(RepeatVector(sequence_length))
        model.add(LSTM(256, return_sequences=True))
    elif cell_type == 'GRU':
        model.add(GRU(256, input_shape=(sequence_length, vocab_size)))
        model.add(RepeatVector(sequence_length))
        model.add(GRU(256, return_sequences=True))
    model.add(Dropout(0.3))
    model.add(TimeDistributed(Dense(vocab_size, activation='softmax')))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])
    return model

In [37]:
results = {}
histories = {}

for cell in ['RNN', 'LSTM', 'GRU']:
    print(f"\nTraining {cell}...")
    model = build_seq2seq(cell)
    history = model.fit(X_train, y_train, epochs=20, batch_size=128,
                        validation_data=(X_val, y_val), verbose=0)
    loss, acc = model.evaluate(X_val, y_val, verbose=0)
    print(f"{cell} Accuracy: {acc*100:.2f}%")
    results[cell] = acc
    histories[cell] = history


Training RNN...
RNN Accuracy: 99.65%

Training LSTM...
LSTM Accuracy: 86.64%

Training GRU...
GRU Accuracy: 80.58%


*Analysis:*

| Model | Accuracy (%) |
| ----- | ------------ |
| RNN   | **99.65**    |
| LSTM  | 86.64        |
| GRU   | 80.58        |


**Conclusion**

In this sequence-to-sequence reversal task with short input sequences, the **Simple RNN significantly outperformed both LSTM and GRU**, achieving an impressive **99.65% validation accuracy**. This result, while counterintuitive given the usual superiority of LSTM/GRU on complex tasks, highlights that **model selection should be based on the nature and complexity of the task**.

* Since the task involves **short-term dependencies** and is **deterministic**, the Simple RNN was sufficient and efficient.
* LSTM and GRU, despite being more advanced, likely **did not converge well** within the same training configuration due to their complexity and need for fine-tuning.

